In [1]:
import pandas as pd
import json
from transformers import pipeline

In [2]:
df = pd.read_excel('04_Consolidado_infobae.xlsx')
df['eval'] = None

In [3]:
# Load a sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# 1 star: Very negative sentiment.
# 2 stars: Negative sentiment.
# 3 stars: Neutral sentiment.
# 4 stars: Positive sentiment.
# 5 stars: Very positive sentiment.

c:\Users\waldo\miniforge3\envs\uni\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
# for index, row in df.iterrows():
#     if row['eval'] is not None:
#         print(f'Skipped: {index}')
#         continue

#     try:
#         oraciones = pd.DataFrame({'oracion' : json.loads(row['oraciones'])})
#         oraciones['etiqueta'] = None
#         oraciones['puntaje'] = None
#         oraciones['estrellas'] = None
#     except:
#         continue
    
#     for ind, oracion in oraciones.iterrows():
#         result = classifier(oracion['oracion'])
#         oraciones.at[ind,'etiqueta'] = result[0]['label']
#         oraciones.at[ind,'estrellas'] = int(result[0]['label'][0])
#         oraciones.at[ind,'puntaje'] = float(result[0]['score'])

#     df.loc[index,'eval'] = oraciones.to_json()
#     print(f'Wrote: {index}', end='\r')

In [4]:
from concurrent.futures import ThreadPoolExecutor

In [5]:
def process_row(index, row):
    if row['eval'] is not None:
        print(f'Skipped: {index}')
        return

    try:
        oraciones = pd.DataFrame({'oracion' : json.loads(row['oraciones'])})
        oraciones['etiqueta'] = None
        oraciones['puntaje'] = None
        oraciones['estrellas'] = None
    except:
        return
    
    for ind, oracion in oraciones.iterrows():
        result = classifier(oracion['oracion'])
        oraciones.at[ind,'etiqueta'] = result[0]['label']
        oraciones.at[ind,'estrellas'] = int(result[0]['label'][0])
        oraciones.at[ind,'puntaje'] = float(result[0]['score'])

    df.loc[index,'eval'] = oraciones.to_json()
    print(f'Wrote: {index}', end='\r')

In [ ]:
with ThreadPoolExecutor(max_workers=12) as executor:
    futures = [executor.submit(process_row, index, row) for index, row in df.iterrows()]
    for future in futures:
        future.result()

In [7]:
df.to_excel('05_Sentimiento.xlsx')